In [1]:
import os
import sys
import json
from copy import deepcopy
import pickle
import importlib

#### Importing the important generic libraries

In [2]:
import beam_helper.beam_utils as beam_utils
import beam_helper.fetch_use_scores as fetch_use_scores
importlib.reload(beam_utils)

/home2/triansh/miniconda3/envs/adv_search_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PYTORCH DEVICE IS  cuda:0


<module 'beam_helper.beam_utils' from '/home2/triansh/adversarial_attacks_on_search/code/beam_search_code/beam_helper/beam_utils.py'>

### Load all the models needed for the pipeline

#### Initialize Faiss Retriever with relevant document store

In [3]:
import beam_helper.faiss_fetch
importlib.reload(beam_helper.faiss_fetch)

2022-10-23 21:15:37.350256: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-23 21:15:37.561251: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-23 21:15:38.953063: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home2/triansh/miniconda3/envs/adv_search_env/lib/
2022-10-23 21:15:38.953296: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: li

<module 'beam_helper.faiss_fetch' from '/home2/triansh/adversarial_attacks_on_search/code/beam_search_code/beam_helper/faiss_fetch.py'>

In [4]:
EXP_NAME = "exp_with_two_thou_short"

In [5]:
faiss_model = beam_helper.faiss_fetch.FaissRetriever(EXP_NAME)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


#### Import model to calculate perplexity scores of a sentence

In [6]:
from transformers import BertTokenizer, BertForMaskedLM

In [7]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [8]:
print("Device is: ", device)

Device is:  cuda:0


In [9]:
per_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
per_model = BertForMaskedLM.from_pretrained('bert-base-uncased')
per_model.to(device)
per_model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

#### Import Counterfitting model

In [10]:
import beam_helper.synonyms_store
importlib.reload(beam_helper.synonyms_store)

<module 'beam_helper.synonyms_store' from '/home2/triansh/adversarial_attacks_on_search/code/beam_search_code/beam_helper/synonyms_store.py'>

In [11]:
cf_model = beam_helper.synonyms_store.synonyms_index( readymade_dict_path="../../data/all_syns.json")

#### Import unmasking model

In [12]:
from transformers import pipeline

In [13]:
NUM_SUGGESTIONS_WANTED = 10

In [14]:
bert_unmasker = pipeline('fill-mask', model='distilbert-base-uncased', top_k=NUM_SUGGESTIONS_WANTED, device=0)

#### Defining the function used to judge validity of nodes in the Parse Tree of a sentence

In [15]:
def checker_func(label):
    if "NN" in label or "NP" in label:
        return True
    
    if "JJ" in label:
        return True
    if "RB" in label:
        return True
    
    if "VB" in label:
        return True
    
    return False

#### Load the `Berkley Neural Parser`

In [16]:
import benepar, spacy
nlp = spacy.load('en_core_web_md')
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})

#### Import universal sentence encoder model

In [17]:
import tensorflow as tf

try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

import tensorflow_hub as hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"
use_model = hub.load(module_url)

/home2/triansh/miniconda3/envs/adv_search_env/lib/python3.9/site-packages/tensorflow_hub/__init__.py:74: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
/home2/triansh/miniconda3/envs/adv_search_env/lib/python3.9/site-packages/tensorflow_hub/__init__.py:75: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  distutils.version.LooseVersion(required_tensorflow_version)):
2022-10-23 21:16:37.568659: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Reinit module models

In [18]:
importlib.reload(beam_utils)
importlib.reload(beam_helper.faiss_fetch)
beam_utils.reinit_models(use_model, per_model, per_tokenizer,cf_model, bert_unmasker, faiss_model, nlp, checker_func )

Models have been reinitialized


### Send the sample post into the `BeamSearch` clas

#### Important attributes needed:
* query_text: Text of the sentence to be perturbed
* post_id: ID of the source document from which the query was derived

In [19]:
sample_input = {
    "ecir_id": 710,
    "query_text": "I bought a tablet about a month ago and since then I have returned it to the store 2 times and gonna return it a third time now and ask for another swap.",
    "post_id": "wkfgt8",
    "query_id": 0
}

#### Decide the parameters

In [20]:
# a lot other parameters can be added as well. Inspect `BeamSearch` c
params_to_send = {
            "query_obj":deepcopy(sample_input), 
            "MAX_DEPTH":3,
            "ALPHA_VAL":0.8, 
}


#### Initialize the beam search class

In [21]:
beam_obj = beam_utils.BeamSearch(**params_to_send)

All params have been initialized
PARENT LEAF CONTENTS IS EMPTY


/home2/triansh/miniconda3/envs/adv_search_env/lib/python3.9/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


#### Start beam search

In [22]:
beam_obj.run_beam_search()

Expanding nodes of depth: 0 to generate nodes for depth: 1


/home2/triansh/miniconda3/envs/adv_search_env/lib/python3.9/site-packages/transformers/pipelines/base.py:1036: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Number of nodes to rank is:  24
Num candidates in the pool for level 1: 60
[after filter] Num candidates in the pool for level 1: 30


/home2/triansh/miniconda3/envs/adv_search_env/lib/python3.9/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


###############  DEPTH DONE ###################################
Expanding nodes of depth: 1 to generate nodes for depth: 2


/home2/triansh/miniconda3/envs/adv_search_env/lib/python3.9/site-packages/transformers/pipelines/base.py:1036: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Number of nodes to rank is:  23
Number of nodes to rank is:  23
Number of nodes to rank is:  23
Number of nodes to rank is:  23
Number of nodes to rank is:  23
Number of nodes to rank is:  23
Number of nodes to rank is:  23
Number of nodes to rank is:  23
Number of nodes to rank is:  22
Number of nodes to rank is:  24
Num candidates in the pool for level 2: 590
[after filter] Num candidates in the pool for level 2: 244
###############  DEPTH DONE ###################################
Expanding nodes of depth: 2 to generate nodes for depth: 3
Number of nodes to rank is:  21
Number of nodes to rank is:  21
Number of nodes to rank is:  21
Number of nodes to rank is:  21
Number of nodes to rank is:  21
Number of nodes to rank is:  22
Number of nodes to rank is:  22
Number of nodes to rank is:  22
Number of nodes to rank is:  22
Number of nodes to rank is:  22
Num candidates in the pool for level 3: 590
[after filter] Num candidates in the pool for level 3: 345
###############  DEPTH DONE ###

In [23]:
ans = beam_obj.fetch_consolidated_results()

In [24]:
with open("sample_output.json",'w') as fd:
    json.dump(ans,fd, indent=2)

#### Let's output the best result for each level of the beam search tree

In [25]:
for curr_level in range(1, len(ans['level_history'])):
    print("Current level is: ", curr_level )
    print("Best pertrubation upto this level was: ")
    best_result_yet = ans['level_history'][curr_level]['best_rank']
    
    print(json.dumps({"perturbed_sentence": best_result_yet['sentence'], "similarity with original sentence":
                     best_result_yet['use_score'],"rank_of_source_document":best_result_yet['q_scores']['best_doc_rank']}, indent=4))
    print("#################################")

Current level is:  1
Best pertrubation upto this level was: 
{
    "perturbed_sentence": "I bought a tablet about a month ago and since then I have returned it to the store 2 times and gon na return it a third time now and ask for another exchanging .",
    "similarity with original sentence": 0.9056332111358643,
    "rank_of_source_document": 8
}
#################################
Current level is:  2
Best pertrubation upto this level was: 
{
    "perturbed_sentence": "I bought a tablet about a month ago and since then I have returned it to the boutique 2 times and gon na return it a third time now and ask for payment .",
    "similarity with original sentence": 0.8028960824012756,
    "rank_of_source_document": 18
}
#################################
Current level is:  3
Best pertrubation upto this level was: 
{
    "perturbed_sentence": "I bought a tablet about a month ago and since then I have returned it to the boutique 2 times and would return it a third time now and ask for anothe